
## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/tips.csv"
file_type = "csv"

df=spark.read.csv(file_location,header=True,inferSchema=True)
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [0]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [0]:
#handling categorical vals
from pyspark.ml.feature import StringIndexer
indexer=StringIndexer(inputCols=["sex","smoker","day","time"],outputCols=["sex_in","smoker_in","day_in","time_in"])
df=indexer.fit(df).transform(df)
df.show()

+----------+----+------+------+---+------+----+------+---------+------+-------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_in|smoker_in|day_in|time_in|
+----------+----+------+------+---+------+----+------+---------+------+-------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|   1.0|      0.0|   1.0|    0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|   0.0|      0.0|   1.0|    0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|   0.0|      0.0|   1.0|    0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|   0.0|      0.0|   1.0|    0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|   1.0|      0.0|   1.0|    0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|   0.0|      0.0|   1.0|    0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|   0.0|      0.0|   1.0|    0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|   0.0|      0.0|   1.0|    0.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|   0.0|      0.0|   1.0|    0.0|
|     14.78|3.23|  Male|    No|Sun|Dinne

In [0]:
df.columns

Out[5]: ['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_in',
 'smoker_in',
 'day_in',
 'time_in']

In [0]:
from pyspark.ml.feature import VectorAssembler
featureass=VectorAssembler(inputCols=['tip','size','sex_in','smoker_in','day_in','time_in'],outputCol="Independent features")
output=featureass.transform(df)
output.select('Independent features').show()

+--------------------+
|Independent features|
+--------------------+
|[1.01,2.0,1.0,0.0...|
|[1.66,3.0,0.0,0.0...|
|[3.5,3.0,0.0,0.0,...|
|[3.31,2.0,0.0,0.0...|
|[3.61,4.0,1.0,0.0...|
|[4.71,4.0,0.0,0.0...|
|[2.0,2.0,0.0,0.0,...|
|[3.12,4.0,0.0,0.0...|
|[1.96,2.0,0.0,0.0...|
|[3.23,2.0,0.0,0.0...|
|[1.71,2.0,0.0,0.0...|
|[5.0,4.0,1.0,0.0,...|
|[1.57,2.0,0.0,0.0...|
|[3.0,4.0,0.0,0.0,...|
|[3.02,2.0,1.0,0.0...|
|[3.92,2.0,0.0,0.0...|
|[1.67,3.0,1.0,0.0...|
|[3.71,3.0,0.0,0.0...|
|[3.5,3.0,1.0,0.0,...|
|(6,[0,1],[3.35,3.0])|
+--------------------+
only showing top 20 rows



In [0]:
finaldata=output.select("Independent features","total_bill")
finaldata.show()

+--------------------+----------+
|Independent features|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
|[3.5,3.0,0.0,0.0,...|     21.01|
|[3.31,2.0,0.0,0.0...|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|[4.71,4.0,0.0,0.0...|     25.29|
|[2.0,2.0,0.0,0.0,...|      8.77|
|[3.12,4.0,0.0,0.0...|     26.88|
|[1.96,2.0,0.0,0.0...|     15.04|
|[3.23,2.0,0.0,0.0...|     14.78|
|[1.71,2.0,0.0,0.0...|     10.27|
|[5.0,4.0,1.0,0.0,...|     35.26|
|[1.57,2.0,0.0,0.0...|     15.42|
|[3.0,4.0,0.0,0.0,...|     18.43|
|[3.02,2.0,1.0,0.0...|     14.83|
|[3.92,2.0,0.0,0.0...|     21.58|
|[1.67,3.0,1.0,0.0...|     10.33|
|[3.71,3.0,0.0,0.0...|     16.29|
|[3.5,3.0,1.0,0.0,...|     16.97|
|(6,[0,1],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [0]:
# training the model
from pyspark.ml.regression import LinearRegression
test_data,train_data=finaldata.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent features',labelCol='total_bill')
regressor=regressor.fit(train_data)


In [0]:
regressor.coefficients

Out[19]: DenseVector([1.1439, 5.3446, -2.2033, 4.4604, 1.0908, -2.6246])

In [0]:
regressor.intercept

Out[20]: 1.5705803767138033

In [0]:
# prediction
predresult=regressor.evaluate(test_data)

In [0]:
predresult.predictions.show()

+--------------------+----------+------------------+
|Independent features|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[1.25,2.0])|     10.07|13.689667124014218|
|(6,[0,1],[1.25,2.0])|     10.51|13.689667124014218|
|(6,[0,1],[1.45,2.0])|      9.55|13.918446701030707|
|(6,[0,1],[1.47,2.0])|     10.77|13.941324658732356|
|(6,[0,1],[1.97,2.0])|     12.02| 14.51327360127358|
| (6,[0,1],[2.0,2.0])|     13.37|14.547590537826053|
| (6,[0,1],[2.0,3.0])|     16.31| 19.89219773329973|
|(6,[0,1],[2.01,2.0])|     20.23|14.559029516676878|
|(6,[0,1],[2.24,3.0])|     16.04| 20.16673322571952|
|(6,[0,1],[2.31,3.0])|     18.69| 20.24680607767529|
| (6,[0,1],[2.5,4.0])|     18.35|25.808753871314632|
| (6,[0,1],[3.0,2.0])|      14.0|  15.6914884229085|
| (6,[0,1],[3.0,4.0])|     20.45| 26.38070281385586|
|(6,[0,1],[3.18,2.0])|     19.82|15.897390042223341|
|(6,[0,1],[3.27,2.0])|     17.78| 16.00034085188076|
|(6,[0,1],[3.35,3.0])|     20.65|21.4364598781

In [0]:
# performance metrics
predresult.r2

Out[23]: 0.5151908806248313

In [0]:
predresult.meanAbsoluteError

Out[24]: 4.899710442942955

In [0]:
predresult.meanSquaredError

Out[25]: 38.34385195257857